In [44]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [45]:
# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [46]:
# from openpyxl import load_workbook
# import pandas as pd
# from pathlib import Path
# src_file = src_file = Path.cwd() / 'GEO-Hi-C.xlsx'
# wb = load_workbook(filename = src_file)
# wb.sheetnames
# sheet = wb['METADATA TEMPLATE ']
# # sheet
# sheet.tables.keys()

### Get files to calculate check sums for (HiC)

In [47]:
import os
import pandas as pd
from pathlib import Path
src_file = Path.cwd() / 'GEO-Hi-C-modif.xlsx'

# df1 = pd.read_excel(
#      os.path.join(APP_PATH, "Data", "aug_latest.xlsm"),
#      engine='openpyxl',
# )

df = pd.read_excel(src_file, header=106, usecols='A:D',engine='openpyxl',nrows=135-107)
df = df.rename(columns={"Unnamed: 3":"path"})
df["path_proc"] = df["path"].str.rstrip("/")+"/"+df["file name"]
# # don't have to save it every time
# df["path_proc"].to_csv("hic_mcool.tsv",sep="\t",index=False,header=None)
# ! scp hic_mcool.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

#######################
# run our little nf pipeline to calculate a bunch of md5-s specified in the hic_mcool.tsv file
####################
# bsub -W 04:00 -oo screen.out -eo screen.err "nextflow run geo.nf --foo screen_fastq"

####
# now deal with the results ...
####
# ! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/hic_mcool.md5 ./
df_res = pd.read_csv("hic_mcool.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path_proc",right_on=0)[1]
df[["file name","file type","file checksum","path"]].to_csv("GEO_FINAL_hic_mcool.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_hic_mcool.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/

GEO_FINAL_hic_mcool.tsv                       100% 4919     8.6KB/s   00:00    


In [48]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/Hi-C/"+df["file name"]
df[["path_proc","dest"]].to_csv("GEO_upload_hic_processed.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_hic_processed.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# # # ! ssh ghpcc bsub 
# ! ssh ghpcc bsub -q long -W 10:00 -oo hicpup.out -eo hicpup.err "bash geo_upload.sh GEO_upload_hic_processed.ssv"

GEO_upload_hic_processed.ssv                  100% 5465    26.0KB/s   00:00    


In [49]:
df = pd.read_excel(src_file, header=138, usecols='A:G',engine='openpyxl',nrows=327-138-1)
df = df.rename(columns={"Unnamed: 6":"path"})
# print(df.loc[0,"path"])
# df["path"].to_csv("hic_fastq.tsv",sep="\t",index=False,header=None)
# ! scp hic_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/hic_fastq.md5 ./
df_res = pd.read_csv("hic_fastq.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path",right_on=0)[1]
df.to_csv("GEO_FINAL_hic_fastq.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_hic_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/


hic_fastq.md5                                 100%   34KB  62.4KB/s   00:00    
GEO_FINAL_hic_fastq.tsv                       100%   48KB  67.0KB/s   00:00    


In [50]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/Hi-C/"+df["file name"]+".fastq.gz"
df[["path","dest"]].to_csv("GEO_upload_hic_fastq.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_hic_fastq.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

# # # file name
# ! ssh ghpcc bsub -q long -W 96:00 -oo hicfup.out -eo hicfup.err "bash geo_upload.sh GEO_upload_hic_fastq.ssv"

GEO_upload_hic_fastq.ssv                      100%   42KB 224.9KB/s   00:00    


In [65]:
df["dest"] = "incoming/"+df["file name"]+".fastq.gz"
i = 1
fp = open(f"SRAKA{i//2}_upload_hic_fastq.ssv","w")
for x,y in df[["path","dest"]].itertuples(index=False):
    fp.write(x)
    fp.write("\n")
    fp.write(y)
    fp.write("\n")
    if (i % 2) == 0:
        fp.close()
        fn = f"SRAKA{i//2}_upload_hic_fastq.ssv"
        fp = open(fn,"w")
    i = i+1
fp.close()
# remove last one if empty ...
_l, = ! wc -l {fn}|cut -f1 -d" "
if not int(str(_l)):
    ! rm {fn}

# fp = open(f"SRAKA{i//2}_upload_hic_fastq.ssv","w")

# ! scp SRA_upload_hic_fastq.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# # ! ssh ghpcc bsub 
# ! ssh ghpcc bsub -q long -W 10:00 -oo chippup.out -eo chippup.err "bash geo_upload.sh SRA_upload_hic_fastq.ssv"

In [73]:
%%bash
for f in SRAKA*_upload_hic_fastq.ssv
do
  scp $f ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
done


In [ ]:
%%bash
for f in SRAKA*_upload_hic_fastq.ssv
do
  /home/sv49w/.aspera/connect/bin/ascp -i ~/.ssh/sra-4.ssh.priv -QT -k1 -l200m --file-pair-list=$f --mode=send --user=asp-sra --host=upload.ncbi.nlm.nih.gov .
done


bsub -W 120:00 -q long -oo sra.out -eo sra.err "/home/sv49w/.aspera/connect/bin/ascp  -i ~/.ssh/sra-4.ssh.priv -QT -l200m --file-pair-list=/nl/umw_job_dekker/users/sv49w/ALV2021/SRA_upload_hic_fastq.ssv --mode=send --user=asp-sra --host=upload.ncbi.nlm.nih.gov ." 

### Get files to calculate check sums for (ChIP-seq)

In [10]:
src_file = Path.cwd() / 'GEO-ChIP-seq.xlsx'
df = pd.read_excel(src_file, header=71, usecols='A:D',engine='openpyxl',nrows=102-72)
df = df.rename(columns={"Unnamed: 3":"path"})
df["path_proc"] = df["path"].str.rstrip("/")+"/"+df["file name"]
df["path_proc"].to_csv("chip_processed.tsv",sep="\t",index=False,header=None)
# ! scp chip_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/chip_processed.md5 ./
df_res = pd.read_csv("chip_processed.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path_proc",right_on=0)[1]
df[["file name","file type","file checksum","path"]].to_csv("GEO_FINAL_chip_processed.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_chip_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/

chip_processed.md5                            100% 4493    14.3KB/s   00:00    
GEO_FINAL_chip_processed.tsv                  100% 4757   224.2KB/s   00:00    


In [11]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/ChIP-seq/"+df["file name"]
df[["path_proc","dest"]].to_csv("GEO_upload_chip_processed.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_chip_processed.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# # ! ssh ghpcc bsub 
# ! ssh ghpcc bsub -q long -W 10:00 -oo chippup.out -eo chippup.err "bash geo_upload.sh GEO_upload_chip_processed.ssv"

GEO_upload_chip_processed.ssv                 100% 5586   144.2KB/s   00:00    


In [23]:
df["dest"] = "test/TEZting_"+df["file name"]
with open("SRA_upload_chip_processed.ssv","w") as fp:
    for i,x,y in df[["path_proc","dest"]].itertuples():
        fp.write(x)
        fp.write("\n")
        fp.write(y)
        fp.write("\n")
    
    #.to_csv("SRA_upload_chip_processed.ssv",sep=" ",index=False,header=None)
! scp SRA_upload_chip_processed.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# # ! ssh ghpcc bsub 
# ! ssh ghpcc bsub -q long -W 10:00 -oo chippup.out -eo chippup.err "bash geo_upload.sh SRA_upload_chip_processed.ssv"

SRA_upload_chip_processed.ssv                 100% 4896    13.9KB/s   00:00    


In [112]:
df = pd.read_excel(src_file, header=105, usecols='A:G',engine='openpyxl',nrows=150-106)
df = df.rename(columns={"Unnamed: 6":"path"})
# df
df["path"].to_csv("chip_fastq.tsv",sep="\t",index=False,header=None)
! scp chip_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/chip_fastq.md5 ./
df_res = pd.read_csv("chip_fastq.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path",right_on=0)[1]
df.to_csv("GEO_FINAL_chip_fastq.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_chip_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/
# ! ssh ghpcc md5sum {df.loc[23,"path"]}

chip_fastq.tsv                                100% 6832   296.1KB/s   00:00    
chip_fastq.md5                                100% 8284    79.0KB/s   00:00    
GEO_FINAL_chip_fastq.tsv                      100%   12KB 398.3KB/s   00:00    


In [114]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/ChIP-seq/"+df["file name"]+".fastq.gz"
df[["path","dest"]].to_csv("GEO_upload_chip_fastq.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_chip_fastq.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

# # # file name
# ! ssh ghpcc bsub -q long -W 36:00 -oo chipfup.out -eo chipfup.err "bash geo_upload.sh GEO_upload_chip_fastq.ssv"

GEO_upload_chip_fastq.ssv                     100%   10KB  67.3KB/s   00:00    


### Get files to calculate check sums for (RNA-seq)

In [97]:
# 90 135
src_file = Path.cwd() / 'GEO-RNA-seq.xlsx'
df = pd.read_excel(src_file, header=90, usecols='A:D',engine='openpyxl',nrows=135-91)
df = df.rename(columns={"Unnamed: 3":"path"})
df["path_proc"] = df["path"].str.rstrip("/")+"/"+df["file name"]
# df["path_proc"].to_csv("rna_processed.tsv",sep="\t",index=False,header=None)
# ! scp rna_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/rna_processed.md5 ./
df_res = pd.read_csv("rna_processed.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path_proc",right_on=0)[1]
df[["file name","file type","file checksum","path"]].to_csv("GEO_FINAL_rna_processed.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_rna_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/
# ! ssh ghpcc md5sum {df.loc[20,"path_proc"]}

rna_processed.md5                             100% 5472   153.6KB/s   00:00    
GEO_FINAL_rna_processed.tsv                   100% 5788    97.9KB/s   00:00    


In [100]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/RNA-seq/"+df["file name"]
df[["path_proc","dest"]].to_csv("GEO_upload_rna_processed.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_rna_processed.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# ! ssh ghpcc bsub 
! ssh ghpcc bsub -q long -W 08:00 -oo rnaup.out -eo rnaup.err "bash geo_upload.sh GEO_upload_rna_processed.ssv"

GEO_upload_rna_processed.ssv                  100% 6402   256.5KB/s   00:00    
bash: bsub: command not found


In [101]:
df = pd.read_excel(src_file, header=138, usecols='A:G',engine='openpyxl',nrows=219-139)
df = df.rename(columns={"Unnamed: 6":"path"})
df["path"].to_csv("rna_fastq.tsv",sep="\t",index=False,header=None)
! scp rna_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/rna_fastq.md5 ./
df_res = pd.read_csv("rna_fastq.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path",right_on=0)[1]
df.to_csv("GEO_FINAL_rna_fastq.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_rna_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/
# ! ssh ghpcc md5sum {df.loc[23,"path"]}

rna_fastq.tsv                                 100%   13KB 547.1KB/s   00:00    
rna_fastq.md5                                 100%   15KB 305.9KB/s   00:00    
GEO_FINAL_rna_fastq.tsv                       100%   22KB 509.3KB/s   00:00    


In [106]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/RNA-seq/"+df["file name"]+".fastq.gz"
df[["path","dest"]].to_csv("GEO_upload_rna_fastq.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_rna_fastq.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

# # file name
# ! ssh ghpcc bsub -q long -W 18:00 -oo rnaup.out -eo rnaup.err "bash geo_upload.sh GEO_upload_rna_fastq.ssv"

GEO_upload_rna_fastq.ssv                      100%   20KB 221.9KB/s   00:00    


### Get files to calculate check sums for (CRISPR Screen sequencing data)

In [4]:

src_file = Path.cwd() / 'GEO-CRISPR-screen.xlsx'
df = pd.read_excel(src_file, header=86, usecols='A:D',engine='openpyxl',nrows=89-87)
df = df.rename(columns={"Unnamed: 3":"path"})
df["path_proc"] = df["path"].str.rstrip("/")+"/"+df["file name"]
df["path_proc"].to_csv("screen_processed.tsv",sep="\t",index=False,header=None)
! scp screen_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

    
# just a couple of small files ! no need to run nextflow for these md5s ...
md5 = []
for fp in df["path_proc"]:
    #     print(fp)
    _md5, = ! ssh ghpcc md5sum {fp} | cut -f1 -d' '
    md5.append(_md5)

# upload straight to the ghpcc ...
df["file checksum"] = md5  
df[["file name","file type","file checksum","path"]].to_csv("GEO_FINAL_screen_processed.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_screen_processed.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/

NameError: name 'Path' is not defined

In [93]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/"+df["file name"]
df[["path_proc","dest"]].to_csv("GEO_upload_screen_processed.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_screen_processed.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
# ! ssh ghpcc bsub 


GEO_upload_screen_processed.ssv               100%  312    16.9KB/s   00:00    


In [95]:
df = pd.read_excel(src_file, header=92, usecols='A:G',engine='openpyxl',nrows=131-93)
df = df.rename(columns={"Unnamed: 6":"path"})
df["path"].to_csv("screen_fastq.tsv",sep="\t",index=False,header=None)
! scp screen_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/
    
####
# now deal with the results ...
####
! scp ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/screen_fastq.md5 ./
df_res = pd.read_csv("screen_fastq.md5",sep=" ",header=None,engine="python")
df["file checksum"] = pd.merge(df,df_res,left_on="path",right_on=0)[1]
df.to_csv("GEO_FINAL_screen_fastq.tsv",sep="\t",index=False,header=None)
! scp GEO_FINAL_screen_fastq.tsv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/GEO/
# # ! ssh ghpcc md5sum {df.loc[23,"path"]}

screen_fastq.tsv                              100% 4294   176.3KB/s   00:00    
screen_fastq.md5                              100% 5548   289.6KB/s   00:00    
GEO_FINAL_screen_fastq.tsv                    100% 7296   320.1KB/s   00:00    


In [96]:
df["dest"] = "uploads/dekkerlab_m7wFeGCY/"+df["file name"]+".fastq.gz"
df[["path","dest"]].to_csv("GEO_upload_screen_fastq.ssv",sep=" ",index=False,header=None)
! scp GEO_upload_screen_fastq.ssv ghpcc:/nl/umw_job_dekker/users/sv49w/ALV2021/

# file name

GEO_upload_screen_fastq.ssv                   100% 6080   188.2KB/s   00:00    


In [105]:
# !ssh ghpcc ls /nl/umw_job_dekker/users/av90w/final-geo
# ncftpput -F -C -z -u geoftp -p "rebUzyi1" ftp-private.ncbi.nlm.nih.gov /nl/umw_job_dekker/users/av90w/final-geo/GEO-RNA-seq.xlsx uploads/dekkerlab_m7wFeGCY/RNA-seq/GEO-RNA-seq.xlsx
# ncftpput -F -C -z -u geoftp -p "rebUzyi1" ftp-private.ncbi.nlm.nih.gov /nl/umw_job_dekker/users/av90w/final-geo/GEO-ChIP-seq.xlsx uploads/dekkerlab_m7wFeGCY/ChIP-seq/GEO-ChIP-seq.xlsx
# ncftpput -F -C -z -u geoftp -p "rebUzyi1" ftp-private.ncbi.nlm.nih.gov /nl/umw_job_dekker/users/av90w/final-geo/GEO-Hi-C.xlsx uploads/dekkerlab_m7wFeGCY/Hi-C/GEO-Hi-C.xlsx

GEO-ChIP-seq.xlsx
GEO-CRISPR-screen.xlsx
GEO-Hi-C.xlsx
GEO-RNA-seq.xlsx


### out little script to do geo uploading with the 2-column source/destination file as input ...

```sh
while read src dest; do
    echo "uploading $src to $dest ..."
    ncftpput -F -C -z -u geoftp -p "rebUzyi1" ftp-private.ncbi.nlm.nih.gov $src $dest
done < $1
```

message to geo 

```
Part 1 out of 4 of a SuperSeries. Title of SuperSeries is 'Three types of chromatin boundaries define a cohesin traffic pattern needed for gene functions'
```

```
Part 2 out of 4 of a SuperSeries. Title of SuperSeries is 'Three types of chromatin boundaries define a cohesin traffic pattern needed for gene functions'. Part 1 was assigned GSE180657
```

Directory
uploads/dekkerlab_m7wFeGCY
User ID
dekkerlab
Submission kind
new
✓ Metadata
✓ Processed data
✓ Raw data
Public release date
2024-07-22
Comment
Part 1 out of 4 of a SuperSeries. Title of SuperSeries is 'Three types of chromatin boundaries define a cohesin traffic pattern needed for gene functions'

Directory
uploads/dekkerlab_m7wFeGCY/ChIP-seq
User ID
dekkerlab
Submission kind
new
✓ Metadata
✓ Processed data
✓ Raw data
Public release date
2024-07-22
Comment
Part 2 out of 4 of a SuperSeries. Title of SuperSeries is 'Three types of chromatin boundaries define a cohesin traffic pattern needed for gene functions'. Part 1 was assigned GSE180657

uploads/dekkerlab_m7wFeGCY/RNA-seq
User ID
dekkerlab
Submission kind
new
✓ Metadata
✓ Processed data
✓ Raw data
Public release date
2024-07-23
Comment
Part 3 out of 4 of the SuperSeries GSE180691 .